## Imports

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging

import sklearn as sklear

import core.dataframe_modeler as cdataf
import core.signal_processing as csigna
import helpers.dbg as dbg
import helpers.env as henv
import helpers.printing as hprint
import instrument_master.kibot.data.load.futures_forward_contracts as vkdlfu
import instrument_master.kibot.data.load.s3_data_loader as vkdls3
import instrument_master.kibot.metadata.load.kibot_metadata as vkmlki

In [ ]:
dbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

## Map contracts to start and end dates

In [ ]:
lfc_hc = vkmlki.KibotHardcodedContractLifetimeComputer(365, 7)

lfc_hc.compute_lifetime("CLJ17")

In [ ]:
lfc_ta = vkmlki.KibotTradingActivityContractLifetimeComputer()

lfc_ta.compute_lifetime("CLJ17")

In [ ]:
symbols = ["ES", "CL", "NG"]
file = "../contracts.csv"


fcl = vkmlki.FuturesContractLifetimes(file, lfc_hc)

In [ ]:
fcl.save(["CL", "NG"])

In [ ]:
data = fcl.load(["NG"])

In [ ]:
data["NG"].head()

## Create continuous contracts

In [ ]:
fcem = vkmlki.FuturesContractExpiryMapper(data)

In [ ]:
fcem.get_nth_contract("NG", "2010-01-01", 1)

In [ ]:
srs = fcem.get_nth_contracts("NG", "2010-01-10", "2010-01-20", freq="B", n=1)

In [ ]:
srs

In [ ]:
kdl = vkdls3.S3KibotDataLoader()

In [ ]:
ffc_obj = vkdlfu.FuturesForwardContracts(kdl)

In [ ]:
ffc_obj._replace_contracts_with_data(srs)

## Combine front and back contracts - price

In [ ]:
contract_df = fcem.get_contracts(
    ["NG" + str(j) for j in range(1, 13)],
    "2010-01-01", "2015-12-31",
    freq="B"
)

In [ ]:
contract_df.head()

In [ ]:
price_df = ffc_obj.replace_contracts_with_data(contract_df, "close")

In [ ]:
price_df.plot()

In [ ]:
dfm = (
    cdataf.DataFrameModeler(df=price_df, oos_start="2013-01-01")
    .compute_ret_0(method="predict")
    .apply_column_transformer(
        transformer_func=csigna.compute_rolling_zscore,
        transformer_kwargs={
            "tau": 10,
            "min_periods": 20,
        },
        col_mode="replace_all",
        method="predict",
    )
)

In [ ]:
dfm.plot_time_series()

In [ ]:
dfm.plot_pca_components(num_components=4)

In [ ]:
dfm.plot_explained_variance()

In [ ]:
res = dfm.apply_residualizer(
    model_func=sklear.decomposition.PCA,
    x_vars=["NG" + str(j) + "_ret_0" for j in range(1, 13)],
    model_kwargs={"n_components": 2},
    method="predict",
).apply_column_transformer(
        transformer_func=csigna.compute_rolling_zscore,
        transformer_kwargs={
            "tau": 10,
            "min_periods": 20,
        },
        col_mode="replace_all",
        method="predict",
)
#.apply_volatility_model(
#    cols=["NG" + str(j) + "_ret_0" for j in range(1, 13)],
#    steps_ahead=2,
#)


In [ ]:
res.df

In [ ]:
res.plot_time_series()

In [ ]:
res.plot_pca_components(num_components=4)

In [ ]:
dfm.plot_explained_variance()

In [ ]:
res.plot_correlation_matrix(mode="ins")

## Combine front and back contracts - volume

In [ ]:
volume_df = ffc_obj.replace_contracts_with_data(contract_df, "vol")

In [ ]:
volume_df

In [ ]:
volume_df.plot(logy=True)